## 1. 사용할 패키지 불러오기

In [1]:
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.nn import L1Loss
import torch.nn.functional as F
from data_gen.data_gen import TotalDatasetGenerator
import torch
import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import timm


/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 데이터 불러오기

In [ ]:
final_data = pd.read_excel('final_data.xlsx')
final_data.head()

## 3. 이미지 별 RGB 평균 및 HSV 평균 변수 추가

In [ ]:
final_data['R'] = 0
final_data['G'] = 0
final_data['B'] = 0
final_data['H'] = 0
final_data['S'] = 0
final_data['V'] = 0

for i in range(len(final_data['이미지경로'])):
    if i % 500 == 0:
        print(i)
    bgr = cv2.imread(final_data['이미지경로'][i])
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    

    final_data['R'][i] = np.mean(bgr[:, :, 2][bgr[:, :, 2] != 255])
    final_data['G'][i] = np.mean(bgr[:, :, 1][bgr[:, :, 1] != 255])
    final_data['B'][i] = np.mean(bgr[:, :, 0][bgr[:, :, 0] != 255])
    final_data['H'][i] = np.mean(hsv[:, :, 0][hsv[:, :, 0] != 255])
    final_data['S'][i] = np.mean(hsv[:, :, 1][hsv[:, :, 1] != 255])
    final_data['V'][i] = np.mean(hsv[:, :, 2][hsv[:, :, 2] != 255])
    

In [ ]:
final_data.to_excel('final_data_rgb_hsv.xlsx', index = False, encoding = 'euc-kr')

## 4. 이미지외 변수 및 이미지 모두활용한 분석

### (1) Dataset 생성

In [11]:
final_data = pd.read_excel('final_data_rgb_hsv.xlsx')

In [12]:
all_variable = ['작가생존여부_사망', '작가생존여부_생존', '작가생존여부_알수없음', '판매계절_가을', '판매계절_겨울', '판매계절_봄',
       '판매계절_여름', '재료_견본채색', '재료_기타', '재료_브론즈', '재료_비단에수묵담채', '재료_석판화',
       '재료_실크스크린', '재료_알수없음', '재료_오프셋석판화', '재료_종에이수묵담채', '재료_종이에먹', '재료_종이에수묵',
       '재료_종이에수묵담채', '재료_종이에수묵채색', '재료_종이에수채', '재료_종이에유채', '재료_지본묵서',
       '재료_지본수묵', '재료_지본채색', '재료_캔버스에아크릴', '재료_캔버스에유채', '재료_캔버스에혼합재료',
       '판매처_꼬모옥션', '판매처_마이아트옥션', '판매처_서울옥션', '판매처_아이옥션', '판매처_에이옥션', '판매처_칸옥션',
       '판매처_케이옥션', '판매처_헤럴드아트데이', '가로', '세로', '작품 판매 횟수', '이미지경로', 'R', 'G', 'B', 'H', 'S', 'V']

table_variable = ['작가생존여부_사망', '작가생존여부_생존', '작가생존여부_알수없음', '판매계절_가을', '판매계절_겨울', '판매계절_봄',
       '판매계절_여름', '재료_견본채색', '재료_기타', '재료_브론즈', '재료_비단에수묵담채', '재료_석판화',
       '재료_실크스크린', '재료_알수없음', '재료_오프셋석판화', '재료_종에이수묵담채', '재료_종이에먹', '재료_종이에수묵',
       '재료_종이에수묵담채', '재료_종이에수묵채색', '재료_종이에수채', '재료_종이에유채', '재료_지본묵서',
       '재료_지본수묵', '재료_지본채색', '재료_캔버스에아크릴', '재료_캔버스에유채', '재료_캔버스에혼합재료',
       '판매처_꼬모옥션', '판매처_마이아트옥션', '판매처_서울옥션', '판매처_아이옥션', '판매처_에이옥션', '판매처_칸옥션',
       '판매처_케이옥션', '판매처_헤럴드아트데이', '가로', '세로', '작품 판매 횟수', 'R', 'G', 'B', 'H', 'S', 'V']

In [13]:
X = final_data.loc[:, all_variable]
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X.loc[:, ['R', 'G', 'B', 'H', 'S', 'V']] = min_max_scaler.fit_transform(X.loc[:, ['R', 'G', 'B', 'H', 'S', 'V']] )
X.head()


y = np.log10(final_data['판매가격'])

In [14]:
train_dataset, test_dataset, train_target, test_target = train_test_split(X, y, train_size = 0.8, random_state = 1004)

### (2) 이미지 데이터와 이미지외 변수 데이터로 분할

In [15]:
train_image = train_dataset['이미지경로']
train_table = train_dataset.loc[:, table_variable]
train_table = np.array(train_table)

test_image = test_dataset['이미지경로']
test_table = test_dataset.loc[:, table_variable]
test_table = np.array(test_table)

### (3) Dataloader 생성

In [16]:
train_dataset_generator = TotalDatasetGenerator(list(train_image), train_table, list(train_target), batch_size = 64, phase = 'train')
train_dataloader = train_dataset_generator.dataloader()

test_dataset_generator = TotalDatasetGenerator(list(test_image), test_table, list(test_target), batch_size = 1, phase = 'test')
test_dataloader = test_dataset_generator.dataloader()

### (4) Model 생성

In [6]:
class TotalModel(nn.Module):
    def __init__(self):
        super(TotalModel, self).__init__()
        self.vision_model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=64)
        
        self.fc11 = nn.Linear(45, 45)
        self.fc12 = nn.Linear(45, 64)

        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 1)
        
    def forward(self, image, table):
        x1 = self.vision_model(image)
        x2 = F.relu(self.fc11(table))
        x2 = F.relu(self.fc12(x2))
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [7]:
total_model = TotalModel()

### (3) 학습 파라미터 지정

In [8]:
epoch = 10
learning_rate = 0.001
weight_decay = 0.0001
result_dir = './result/'

### (4) Loss, Optimizer 생성

In [9]:
# get loss function from LossFactory
loss_fn = L1Loss()

# get optimizer from OptimizerFactory
optimizer = Adam(params = total_model.parameters(),
                lr=learning_rate,
                weight_decay = weight_decay)

### (5) 학습

In [17]:
print("{} start training!".format('vit'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
total_model.to(device)
min_valid_loss = np.inf

# training
for e in range(epoch):
    train_loss = 0.0
    total_model.train()   
    for data in tqdm(train_dataloader['train']):
        if torch.cuda.is_available():
            images, table, labels = data['image'].float().to(device),  data['table'].float().to(device), data['target'].float().to(device)
        
        optimizer.zero_grad()
        target = total_model(images, table)
        loss = loss_fn(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / len(images)
            
    valid_loss = 0.0
    total_model.eval()  
    for data in tqdm(test_dataloader['test']):
        if torch.cuda.is_available():
            images, table, labels = data['image'].float().to(device),  data['table'].float().to(device), data['target'].float().to(device)
        
        target = total_model(images, table)
        loss = loss_fn(target,labels)
        valid_loss = loss.item() * len(data)

    print("Epoch: {}, Training Loss: {}, Test Loss: {}".format(e+1, train_loss / len(train_dataloader['train']), valid_loss))   
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(total_model.state_dict(), result_dir + 'Best_total_vit_model.pth')    

vit start training!


  0%|          | 0/188 [00:00<?, ?it/s]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
 99%|█████████▉| 187/188 [03:51<00:01,  1.22s/it]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([58])) that is different to the input size (torch.Size([58, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  0%|          | 0/3007 [00:00<?, ?it/s]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:9

Epoch: 1, Training Loss: 0.009673763450453097, Test Loss: 0.34447145462036133
Validation Loss Decreased(inf--->0.344471) 	 Saving The Model


100%|██████████| 3007/3007 [00:47<00:00, 63.86it/s]


Epoch: 2, Training Loss: 0.008334178030398631, Test Loss: 1.0963239669799805


...

Epoch: 8, Training Loss: 0.008187798302404046, Test Loss: 0.753354549407959


100%|██████████| 3007/3007 [00:51<00:00, 58.67it/s]


Epoch: 9, Training Loss: 0.008178597687940391, Test Loss: 0.5353531837463379


100%|██████████| 3007/3007 [00:48<00:00, 61.72it/s]

Epoch: 10, Training Loss: 0.008194174126073478, Test Loss: 0.42096376419067383


### (6) Load Best Model

In [18]:
total_model.load_state_dict(torch.load('result/Best_total_vit_model.pth'))

<All keys matched successfully>

### (7) 성능 평가

#### - 학습 데이터에 대한 성능

In [19]:
print('start prediction')
predictions = []
total_model.to(device)

with torch.no_grad():  
    for data in train_dataloader['train']:
        images, table, labels = data['image'].float().to(device),  data['table'].float().to(device), data['target'].float().to(device)
        total_model.eval()  
        yhat = total_model(images, table)  
        pred = list(yhat.cpu().numpy())
        predictions = predictions + list(np.hstack(pred))

start prediction


In [20]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(train_target, predictions))))
print("R2 Score: {}".format(r2_score(train_target, predictions)))

RMSE: 0.6801387456869593
R2 Score: -7.5023272920971e-05


#### - 테스트 데이터에 대한 성능

In [21]:
print('start prediction')
predictions = []
total_model.to(device)

with torch.no_grad():  
    for data in test_dataloader['test']:
        images, table, labels = data['image'].float().to(device),  data['table'].float().to(device), data['target'].float().to(device)
        total_model.eval()  
        yhat = total_model(images, table)  
        pred = list(yhat.cpu().numpy())
        predictions.append(pred[0][0])

start prediction


In [22]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(test_target, predictions))))
print("R2 Score: {}".format(r2_score(test_target, predictions)))

RMSE: 0.7000380304463465
R2 Score: 0.0008174358609462917
